In [1]:
import psycopg2
import pandas as pd

# Prepare the Data

Run `docker-compose up` to get Postgres up and running.

In [2]:
credentials = "dbname='astronomy' user='user' host='localhost' password='pass'"
conn = psycopg2.connect(credentials)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS Planet;
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  kepler_id INTEGER NOT NULL,
  t_eff INTEGER NOT NULL,
  radius FLOAT NOT NULL,
  PRIMARY KEY (kepler_id)
);

CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL REFERENCES Star(Kepler_ID),
  koi_name VARCHAR(20) NOT NULL,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT NOT NULL,
  radius FLOAT NOT NULL,
  t_eq INTEGER NOT NULL,
  PRIMARY KEY (koi_name)
);


COPY Star (kepler_id, t_eff, radius)
  FROM '/data/data3/stars.csv' CSV;

COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq)
  FROM '/data/data3/planets.csv' CSV;
""")
conn.commit()

# Task 1
Write a query that returns the radius of each star and planet pair whose radii
have a ratio greater than the Sun-to-Earth radius ratio. Order the results in
descending order based on the stellar radii. Use `sun_radius` and
`planet_radius` as attribute aliases for the star and planet radii.

For this problem you will have to join the two tables to find all planets
belonging to a given star and use a condition to select those results which
fulfill the size requirement above.

In [3]:
pd.read_sql("""
SELECT
  Star.radius AS sun_radius,
  Planet.radius AS planet_radius
FROM Star, Planet
WHERE
  Star.kepler_id = Planet.kepler_id AND
  Star.radius > Planet.radius
ORDER BY Star.radius DESC;
""", conn)

,sun_radius,planet_radius
0,1.332,0.65
1,1.029,0.85
2,1.029,0.99
3,0.755,0.58
4,0.755,0.49


# Task 2
Write a query which counts the number of planets in each solar system where
the corresponding stars are larger than our sun (i.e. their radius is larger
than 1).

Your query should return the star's radius and its number of planets, showing
only rows where the number of planets is more than one. Sort the rows in
descending order based on the star radii.

In [4]:
pd.read_sql("""
SELECT Star.radius, COUNT(Planet.koi_name)
FROM Star
JOIN Planet USING (kepler_id)
WHERE Star.radius > 1
GROUP BY Star.kepler_id
HAVING COUNT(Planet.koi_name) > 1
ORDER BY Star.radius DESC;
""", conn)

,radius,count
0,1.332,3
1,1.064,2
2,1.040,2
3,1.029,4


# Task 3
## Prepare Data

In [5]:
cur.execute("""
DROP TABLE IF EXISTS Planet;
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  Kepler_ID INTEGER NOT NULL,
  T_eff INTEGER NOT NULL,
  Radius FLOAT NOT NULL,
  PRIMARY KEY (Kepler_ID)
);

CREATE TABLE Planet (
  Kepler_ID INTEGER NOT NULL,
  KOI_name VARCHAR(20) NOT NULL,
  Kepler_name VARCHAR(20),
  Status VARCHAR(20) NOT NULL,
  Period FLOAT,
  Radius FLOAT,
  T_eq INTEGER,
  PRIMARY KEY (KOI_name)
);

INSERT INTO Star VALUES(2713049,5996,0.956);
INSERT INTO Star VALUES(3114167,5666,0.677);
INSERT INTO Star VALUES(3246984,5735,0.973);
INSERT INTO Star VALUES(3351888,5717,1.057);
INSERT INTO Star VALUES(3641726,5349,0.82);
INSERT INTO Star VALUES(3832474,5485,0.867);
INSERT INTO Star VALUES(3935914,5934,0.893);
INSERT INTO Star VALUES(3940418,5170,0.807);
INSERT INTO Star VALUES(4139816,3887,0.48);
INSERT INTO Star VALUES(4275191,5557,0.781);
INSERT INTO Star VALUES(4476123,5413,0.751);
INSERT INTO Star VALUES(5358241,6079,0.945);
INSERT INTO Star VALUES(5358624,5071,0.788);
INSERT INTO Star VALUES(5456651,4980,0.734);
INSERT INTO Star VALUES(6862328,5796,0.871);
INSERT INTO Star VALUES(6922244,6225,1.451);
INSERT INTO Star VALUES(8395660,5881,1.029);
INSERT INTO Star VALUES(9579641,6391,1.332);
INSERT INTO Star VALUES(10187017,4812,0.755);
INSERT INTO Star VALUES(10480982,6117,0.947);
INSERT INTO Star VALUES(10583066,4536,0.693);
INSERT INTO Star VALUES(10601284,5559,0.806);
INSERT INTO Star VALUES(10666592,6350,1.991);
INSERT INTO Star VALUES(10682541,5339,0.847);
INSERT INTO Star VALUES(10797460,5850,1.04);
INSERT INTO Star VALUES(10811496,5853,0.868);
INSERT INTO Star VALUES(10848459,5795,0.803);
INSERT INTO Star VALUES(10854555,6031,1.046);
INSERT INTO Star VALUES(10872983,6046,0.972);
INSERT INTO Star VALUES(10910878,5126,0.742);
INSERT INTO Star VALUES(10984090,5803,1.073);
INSERT INTO Star VALUES(10987985,5015,0.826);
INSERT INTO Star VALUES(11018648,5588,0.796);
INSERT INTO Star VALUES(11138155,6117,1.025);
INSERT INTO Star VALUES(11153539,6075,0.969);
INSERT INTO Star VALUES(11304958,5468,1.046);
INSERT INTO Star VALUES(11391957,5592,0.782);
INSERT INTO Star VALUES(11403044,6174,1.103);
INSERT INTO Star VALUES(11414511,5653,0.965);
INSERT INTO Star VALUES(11493732,6144,1.091);
INSERT INTO Star VALUES(11507101,5957,0.971);
INSERT INTO Star VALUES(11754553,3898,0.54);
INSERT INTO Star VALUES(11812062,5492,0.812);
INSERT INTO Star VALUES(11818800,5446,0.781);
INSERT INTO Star VALUES(11853255,3741,0.45);
INSERT INTO Star VALUES(11904151,5627,1.056);
INSERT INTO Star VALUES(11918099,4989,0.727);
INSERT INTO Star VALUES(11923270,3672,0.49);
INSERT INTO Star VALUES(11960862,5992,0.989);
INSERT INTO Star VALUES(12070811,5557,0.752);
INSERT INTO Star VALUES(12110942,5880,0.917);
INSERT INTO Star VALUES(12366084,5841,0.931);
INSERT INTO Star VALUES(12470844,5354,0.788);
INSERT INTO Star VALUES(12644822,5795,0.919);

INSERT INTO Planet VALUES(10666592,'K00002.01','Kepler-2b','CONFIRMED',2.204735365,16.39,2025);
INSERT INTO Planet VALUES(11904151,'K00072.01','Kepler-10b','CONFIRMED',0.837491331,1.45,1968);
INSERT INTO Planet VALUES(10187017,'K00082.04','Kepler-102c','CONFIRMED',7.07136076,0.58,723);
INSERT INTO Planet VALUES(10187017,'K00082.05','Kepler-102b','CONFIRMED',5.28695437,0.49,797);
INSERT INTO Planet VALUES(10984090,'K00112.02','Kepler-466c','CONFIRMED',3.709213846,1.24,1236);
INSERT INTO Planet VALUES(9579641,'K00115.01','Kepler-105b','CONFIRMED',5.41220713,3.28,1306);
INSERT INTO Planet VALUES(9579641,'K00115.02','Kepler-105c','CONFIRMED',7.12594591,1.88,1191);
INSERT INTO Planet VALUES(9579641,'K00115.03',NULL,'CANDIDATE',3.4358789,0.65,1519);
INSERT INTO Planet VALUES(8395660,'K00116.01','Kepler-106c','CONFIRMED',13.57076622,2.35,796);
INSERT INTO Planet VALUES(8395660,'K00116.02','Kepler-106e','CONFIRMED',43.84444353,2.58,538);
INSERT INTO Planet VALUES(8395660,'K00116.03','Kepler-106b','CONFIRMED',6.16491696,0.85,1035);
INSERT INTO Planet VALUES(8395660,'K00116.04','Kepler-106d','CONFIRMED',23.9802348,0.99,658);
INSERT INTO Planet VALUES(10601284,'K00749.03','Kepler-226d','CONFIRMED',8.10904671,1.19,799);
INSERT INTO Planet VALUES(10662202,'K00750.01','Kepler-662b','CONFIRMED',21.67697486,1.54,430);
INSERT INTO Planet VALUES(10682541,'K00751.01','Kepler-663b','CONFIRMED',4.99678284,2.7,917);
INSERT INTO Planet VALUES(10811496,'K00753.01',NULL,'CANDIDATE',19.899139805,3462.25,639);
INSERT INTO Planet VALUES(10848459,'K00754.01',NULL,'CANDIDATE',1.736952479,34.04,1404);
INSERT INTO Planet VALUES(10854555,'K00755.01','Kepler-664b','CONFIRMED',2.525593315,2.71,1407);
INSERT INTO Planet VALUES(10872983,'K00756.01','Kepler-228d','CONFIRMED',11.09431923,4.02,835);
INSERT INTO Planet VALUES(10872983,'K00756.02','Kepler-228c','CONFIRMED',4.13443005,3.02,1160);
INSERT INTO Planet VALUES(10872983,'K00756.03','Kepler-228b','CONFIRMED',2.56659092,1.56,1360);
INSERT INTO Planet VALUES(10910878,'K00757.01','Kepler-229c','CONFIRMED',16.06862959,5.27,571);
INSERT INTO Planet VALUES(10910878,'K00757.02','Kepler-229d','CONFIRMED',41.1970874,3.62,417);
INSERT INTO Planet VALUES(10910878,'K00757.03','Kepler-229b','CONFIRMED',6.252964898,2.41,782);
INSERT INTO Planet VALUES(10987985,'K00758.01','Kepler-665b','CONFIRMED',16.01310205,2.86,593);
INSERT INTO Planet VALUES(11018648,'K00759.01','Kepler-230b','CONFIRMED',32.62882975,3791.05,506);
INSERT INTO Planet VALUES(11018648,'K00759.02','Kepler-230c','CONFIRMED',91.77221,2.13,358);
INSERT INTO Planet VALUES(11138155,'K00760.01',NULL,'CANDIDATE',4.959319451,11.88,1128);
INSERT INTO Planet VALUES(11153539,'K00762.01','Kepler-666b','CONFIRMED',4.49876092,2.21,1133);
INSERT INTO Planet VALUES(11304958,'K00764.01','Kepler-667b','CONFIRMED',41.43962808,5.73,516);
INSERT INTO Planet VALUES(11391957,'K00765.01','Kepler-668b','CONFIRMED',8.35390639,2.54,789);
INSERT INTO Planet VALUES(11403044,'K00766.01','Kepler-669b','CONFIRMED',4.125546869,4.46,1244);
INSERT INTO Planet VALUES(11460018,'K00769.01','Kepler-671b','CONFIRMED',4.280958588,2.33,1014);
INSERT INTO Planet VALUES(11465813,'K00771.01',NULL,'CANDIDATE',670.645246,14.41,196);
INSERT INTO Planet VALUES(11493732,'K00772.01',NULL,'CANDIDATE',61.2563443,64.23,505);
INSERT INTO Planet VALUES(11507101,'K00773.01','Kepler-672b','CONFIRMED',38.3774623,2.7,541);
INSERT INTO Planet VALUES(11754553,'K00775.01','Kepler-52c','CONFIRMED',16.38485646,1.81,392);
INSERT INTO Planet VALUES(11754553,'K00775.02','Kepler-52b','CONFIRMED',7.87740709,2.33,500);
INSERT INTO Planet VALUES(11754553,'K00775.03','Kepler-52d','CONFIRMED',36.4451982,1.8,300);
INSERT INTO Planet VALUES(11812062,'K00776.01','Kepler-673b','CONFIRMED',3.728731093,6.27,1022);
INSERT INTO Planet VALUES(11818800,'K00777.01',NULL,'CANDIDATE',40.41958501,8.02,468);
INSERT INTO Planet VALUES(11853255,'K00778.01','Kepler-674b','CONFIRMED',2.243381847,1.32,685);
INSERT INTO Planet VALUES(11918099,'K00780.01','Kepler-675b','CONFIRMED',2.33743801,2.38,1054);
INSERT INTO Planet VALUES(11918099,'K00780.02',NULL,'CANDIDATE',7.2406514,5.32,723);
INSERT INTO Planet VALUES(11923270,'K00781.01','Kepler-676b','CONFIRMED',11.59822172,3.07,400);
INSERT INTO Planet VALUES(11960862,'K00782.01','Kepler-677b','CONFIRMED',6.57531678,5.38,1015);
INSERT INTO Planet VALUES(12020329,'K00783.01','Kepler-678b','CONFIRMED',7.27503724,4.91,833);
INSERT INTO Planet VALUES(12066335,'K00784.01','Kepler-231c','CONFIRMED',19.2715468,1.73,343);
INSERT INTO Planet VALUES(12066335,'K00784.02','Kepler-231b','CONFIRMED',10.06525843,1.61,426);
INSERT INTO Planet VALUES(12070811,'K00785.01','Kepler-679b','CONFIRMED',12.39358604,2.69,680);
INSERT INTO Planet VALUES(12110942,'K00786.01','Kepler-680b','CONFIRMED',3.689926291,1.96,1147);
INSERT INTO Planet VALUES(12366084,'K00787.01','Kepler-232b','CONFIRMED',4.431242593,3.07,1074);
INSERT INTO Planet VALUES(12366084,'K00787.02','Kepler-232c','CONFIRMED',11.37938071,3.74,784);
INSERT INTO Planet VALUES(12404086,'K00788.01','Kepler-681b','CONFIRMED',26.39435646,3.16,491);
INSERT INTO Planet VALUES(12470844,'K00790.01','Kepler-233b','CONFIRMED',8.47237844,2.71,752);
INSERT INTO Planet VALUES(12470844,'K00790.02','Kepler-233c','CONFIRMED',60.4186137,2.72,390);
INSERT INTO Planet VALUES(12644822,'K00791.01','Kepler-682b','CONFIRMED',12.611906672,7.66,753);
INSERT INTO Planet VALUES(4049131,'K00811.01','Kepler-687b','CONFIRMED',20.50586978,3.62,518);
INSERT INTO Planet VALUES(4139816,'K00812.01','Kepler-235b','CONFIRMED',3.34021995,2.18,635);
INSERT INTO Planet VALUES(4139816,'K00812.02','Kepler-235d','CONFIRMED',20.06037454,1.99,350);
INSERT INTO Planet VALUES(4139816,'K00812.03','Kepler-235e','CONFIRMED',46.18415,1.94,265);
INSERT INTO Planet VALUES(4139816,'K00812.04','Kepler-235c','CONFIRMED',7.82501206,1.22,478);
INSERT INTO Planet VALUES(4476123,'K00814.01','Kepler-689b','CONFIRMED',22.36656079,2.45,544);
INSERT INTO Planet VALUES(5358241,'K00829.01','Kepler-53b','CONFIRMED',18.64929678,2.9,700);
INSERT INTO Planet VALUES(5358241,'K00829.02','Kepler-53d','CONFIRMED',9.75193182,2.44,869);
INSERT INTO Planet VALUES(5358241,'K00829.03','Kepler-53c','CONFIRMED',38.5575914,3.57,550);
INSERT INTO Planet VALUES(5358624,'K00830.01','Kepler-428b','CONFIRMED',3.525632561,11.87,955);
INSERT INTO Planet VALUES(5456651,'K00835.01','Kepler-239b','CONFIRMED',11.76305946,2.36,614);
INSERT INTO Planet VALUES(5456651,'K00835.02','Kepler-239c','CONFIRMED',56.2279697,2.19,365);
INSERT INTO Planet VALUES(10875245,'K00117.02','Kepler-107c','CONFIRMED',4.90143807,1.84,1263);
INSERT INTO Planet VALUES(10526549,'K00746.01','Kepler-660b','CONFIRMED',9.27358194,2.52,649);
INSERT INTO Planet VALUES(10583066,'K00747.01','Kepler-661b','CONFIRMED',6.029301321,3.14,685);
INSERT INTO Planet VALUES(10601284,'K00749.01','Kepler-226c','CONFIRMED',5.34955671,2.7,918);
INSERT INTO Planet VALUES(10601284,'K00749.02','Kepler-226b','CONFIRMED',3.94104632,1.59,1017);
INSERT INTO Planet VALUES(2713049,'K00794.01','Kepler-683b','CONFIRMED',2.539183179,1.97,1348);
INSERT INTO Planet VALUES(3342970,'K00800.01','Kepler-234b','CONFIRMED',2.711502579,3.62,1405);
INSERT INTO Planet VALUES(3342970,'K00800.02','Kepler-234c','CONFIRMED',7.21204152,3.51,1015);
INSERT INTO Planet VALUES(3351888,'K00801.01','Kepler-685b','CONFIRMED',1.6255222,9.74,1572);
INSERT INTO Planet VALUES(3453214,'K00802.01',NULL,'CANDIDATE',19.620347388,12.0,605);
INSERT INTO Planet VALUES(3641726,'K00804.01',NULL,'CANDIDATE',9.0293089,2.72,757);
INSERT INTO Planet VALUES(3935914,'K00809.01','Kepler-686b','CONFIRMED',1.594745463,11.77,1540);
INSERT INTO Planet VALUES(3940418,'K00810.01',NULL,'CANDIDATE',4.78300451,2.76,886);
""")
conn.commit()

## Solution
To practise your outer joins, write a query which returns the `kepler_id`,
`t_eff` and `radius` for all stars in the `Star` table which haven't got a
planet as join partner. Order the resulting table based on the `t_eff`
attribute in descending order.

In [6]:
pd.read_sql("""
SELECT Star.kepler_id, Star.t_eff, Star.radius
FROM Star
LEFT OUTER JOIN Planet USING (kepler_id)
WHERE Planet.koi_name IS NULL
ORDER BY Star.t_eff DESC;
""", conn)

,kepler_id,t_eff,radius
0,6922244,6225,1.451
1,10480982,6117,0.947
2,10797460,5850,1.040
3,6862328,5796,0.871
4,3246984,5735,0.973
5,3114167,5666,0.677
6,11414511,5653,0.965
7,4275191,5557,0.781
8,3832474,5485,0.867


# Task 4
## Re-init the Data

In [7]:
cur.execute("""
DROP TABLE IF EXISTS Planet;
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  kepler_id INTEGER NOT NULL,
  t_eff INTEGER NOT NULL,
  radius FLOAT NOT NULL,
  PRIMARY KEY (kepler_id)
);

CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL,
  koi_name VARCHAR(20) NOT NULL,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT NOT NULL,
  radius FLOAT NOT NULL,
  t_eq INTEGER NOT NULL,
  PRIMARY KEY (koi_name)
);


COPY Star (kepler_id, t_eff, radius)
  FROM '/data/data3/stars.csv' CSV;

COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq)
  FROM '/data/data3/planets.csv' CSV;
""")
conn.commit()

## Solution
Write a query which queries both the `Star` and the `Planet` table and
calculates the following quantities:

- the average value of the planets' equilibrium temperature `t_eq`, rounded to
  one decimal place;
- the minimum effective temperature `t_eff` of the stars;
- the maximum value of `t_eff`;
    
Your query should only use those star-planet pairs whose stars have a higher
temperature (`t_eff`) than the average star temperature in the table. Try to
use a subquery to solve this problem!

In [8]:
pd.read_sql("""
SELECT ROUND(AVG(Planet.t_eq), 1), MIN(Star.t_eff), MAX(Star.t_eff)
FROM Star
JOIN Planet USING (kepler_id)
WHERE Star.t_eff > (SELECT AVG(t_eff) FROM Star);
""", conn)

,round,min,max
0,999.1,5492,6391


# Task 6
Write a query which finds the radii of those planets in the `Planet` table
which orbit the five largest stars in the `Star` table.

Your query should return the planet's `koi_name` and `radius` as well as the
corresponding star `radius`.

This problem needs a subquery (to find the largest 5 stars by radius) and
lots of other things you've learned this week! It can be done with, or without,
a join though.

In [9]:
pd.read_sql("""
SELECT
  Planet.koi_name,
  Planet.radius,
  Star.radius
FROM Star
JOIN Planet USING (kepler_id)
WHERE Star.kepler_id IN (
  SELECT kepler_id
  FROM Star
  ORDER BY radius DESC
  LIMIT 5
);
""", conn)

,koi_name,radius,radius
0,K00002.01,16.39,1.991
1,K00010.01,14.83,1.451
2,K00115.01,3.28,1.332
3,K00115.02,1.88,1.332
4,K00115.03,0.65,1.332
5,K00117.02,1.84,1.411
6,K00766.01,4.46,1.103


# Tear down

In [10]:
cur.close()
conn.close()